## Hyperparam

In [1]:
block_size = 140
# max_len = 20
batch_size = 128
n_layer = 8 
n_head = 8 
n_embd = 128
max_epochs = 5

## Import

In [2]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

# from flattenV3Advanced import flattenData

# make deterministic
from minGPT.mingpt.utils import set_seed
set_seed(42)

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.cuda.current_device()

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
import math
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchtext.data.metrics import bleu_score

from minGPT.mingpt.model import GPT, GPTConfig
# from minGPT.mingpt.trainer import Trainer, TrainerConfig
# from minGPT.mingpt.utils import sample
from torch.utils.data.dataloader import DataLoader

from tqdm import tqdm
import random
import pandas as pd
import string
import re
import os
import time

import logging

logger = logging.getLogger(__name__)



from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit import Chem
from rdkit.rdBase import BlockLogs
block = BlockLogs() ## not sure we want to block all but rdkit complain when wrong smiles are sent...

## dataset path
data_path = '/home/teleport/perso/2pc_cleaned.csv'
EXPERIMENT = '20201208_DF_gpt_smiles'
experiment_path = "/opt/data/train_dir/" + EXPERIMENT + "/"
if not os.path.exists(experiment_path):
    os.mkdir(experiment_path)

%matplotlib inline

12/08/2020 15:29:47 - INFO - rdkit -   Enabling RDKit 2020.09.1 jupyter extensions


## Trains and tensorboard

In [3]:
now = datetime.now()
writer = SummaryWriter(experiment_path + '/' + now.strftime("%Y%m%d-%H%M%S") + "/")
from trains import Task
import trains
task = Task.init(project_name='gpt_smiles', task_name=EXPERIMENT)

TRAINS Task: overwriting (reusing) task id=281a4a3e68b14b82bdd1fd8b55b938b2
TRAINS results page: http://172.22.4.157:8080/projects/8f2ac9400a46477683280746b293e09f/experiments/281a4a3e68b14b82bdd1fd8b55b938b2/output/log


## Dataset

In [4]:
data = pd.read_csv(data_path)
data.columns

2020-12-08 15:29:48,682 - trains.Repository Detection - WARNING - Can't get branch information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2020-12-08 15:29:48,691 - trains.Repository Detection - WARNING - Can't get commit information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2020-12-08 15:29:48,699 - trains.Repository Detection - WARNING - Can't get root information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2020-12-08 15:29:48,708 - trains.Repository Detection - WARNING - Can't get status information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2020-12-08 15:29:48,748 - trains.Repository Detection - WARNING - Can't get diff information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2020-12-08 15:29:48,760 - trains.Repository Detection - WARNING - Can't get modified information fo

Index(['smiles', 'mwt', 'logp', 'n_atoms', 'MolFormula', 'l', 'CHNO',
       'l_MolFormula', 'l_chunk', 'train'],
      dtype='object')

In [5]:
data_train = data[data['train'] == True].reset_index()
data_test = data[data['train'] == False].reset_index()
print(len(data))
print(len(data_train))
print(len(data_test))

8659463
8605866
53597


## word list

In [6]:
word_list = ['C','c','N','n','O','o','H','[',']','(',')','-','+','=','@','/','\\','#','$','&']
word_list += list(string.digits)
stoi = { ch:i for i,ch in enumerate(word_list) }
itos = { i:ch for i,ch in enumerate(word_list) }

## Helper functions for checking the smiles and tokenize them, detokenize them

In [7]:
def tokenize_idx(data, idx, stoi, block_size = None, stop_token = '$', split_token = '&'):
    return tokenize_smiles(data['smiles'][idx],data['MolFormula'][idx], stoi, block_size=block_size, stop_token = '$', split_token = '&')

def tokenize_smiles(smiles, molecular_formula, stoi, block_size = None, stop_token = '$', split_token = '&'):
    chunk = molecular_formula + split_token + smiles + stop_token
    if block_size is not None:
        chunk = chunk + stop_token*(block_size - len(chunk))
    data = [stoi[s] for s in chunk]
    return data

def y_to_completion(y, itos):
    return ''.join([itos[int(i)] for i in y])
    

def check_completion(completion, stop_token = '$', split_token = '&'):
    '''
    return a string with the status of the check:
    full_failure: cannot process it at all
    smiles_failure: MolFromSmiles return None
    MolFormula_failure: CalcMolFormula from the smile is different
    success: CalcMolFormula from the smile is same
    '''
    
    try:
        completion = completion.split(split_token)
        MolFormula = completion[0]
        smiles = completion[1].split(stop_token)[0]
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return 'smiles_failure'
        MolFormula_smiles = CalcMolFormula(mol)
        if MolFormula != MolFormula_smiles:
            return 'MolFormula_failure'
        return 'success'
    except:
        return 'full_failure'


In [8]:
y = tokenize_idx(data_test, 0, stoi)
completion = y_to_completion(y, itos)

check_completion(completion)

'success'

In [9]:
completion = 'C23H39N5O&CN(C)c1ccc(CNC(=O)CN2CCC(C)(CN3CCN(C)CC3)CC2)cc1'
check_completion(completion)

'success'

## Dataloader

In [10]:
class SmilesDataset(Dataset):

    def __init__(self, data, block_size, stoi, itos):
        
        self.data = data
        self.stoi = stoi
        self.itos = itos
        self.block_size = block_size
        self.vocab_size = len(self.stoi)
        
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx,verbose=False):
        converted_chunk = tokenize_idx(self.data, idx, self.stoi, self.block_size)
            
        x = torch.tensor(converted_chunk[:-1], dtype=torch.long)
        y = torch.tensor(converted_chunk[1:], dtype=torch.long)
        return x, y
            
train_dataset = SmilesDataset(data_train, block_size, stoi, itos) 
test_dataset = SmilesDataset(data_test, block_size, stoi, itos) 

df = train_dataset.__getitem__(idx = 10,verbose=True)

## Model declaration

In [11]:
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=n_layer, n_head=n_head, n_embd=n_embd)
model = GPT(mconf)
# model.load_state_dict(torch.load(experiment_path + 'model_0.pt'))

12/08/2020 15:29:58 - INFO - minGPT.mingpt.model -   number of parameters: 1.612032e+06


## Sample function

In [12]:
def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[:, [-1]]] = -float('Inf')
    return out

@torch.no_grad()
def sample_clean(model, xs, steps, temperature=1.0, sample=False, top_k=None, itos = None, stop_token = '$',batch_size = 4):
#     block_size = model.get_block_size()
    model.eval()
    xs = sorted(xs, key = lambda x: x.size(1))
    ## add the batch size
    xs = [torch.cat(batch_size*[x]) for x in xs]
    
    ## start by making them the same length
    x = xs[0]
    for i in range(1,len(xs)): # nothing to do if length 1
        while x.size(1) < xs[i].size(1):
            x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
            logits, _ = model(x_cond)
            # pluck the logits at the final step and scale by temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop probabilities to only the top k options
            if top_k is not None:
                logits = top_k_logits(logits, top_k)
            # apply softmax to convert to probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution or take the most likely
            if sample:
                ix = torch.multinomial(probs, num_samples=1)
            else:
                _, ix = torch.topk(probs, k=1, dim=-1)
            # append to the sequence and continue
            x = torch.cat((x, ix), dim=1)
        x= torch.cat((x,xs[i]), dim=0)
    
    
    ## now we can keep going...
    keep_going = True
    while keep_going:
#         print(x.size(1))
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
        logits, _ = model(x_cond)
        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)
        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)
        if x.size(1) == block_size:
            keep_going = False
        if itos is not None:
            sequences_over = 0
            for i in range(x.size(0)):                
                if itos[int(ix[i])] == stop_token:
                    sequences_over += 1
            if sequences_over == x.size(0):
                keep_going = False
    return x

def check_model_output(model, MolFormulas, stop_token = '$', split_token = '&', batch_size = 4):
    results = []
    completions = []
    for i in MolFormulas:        
        data = [stoi[s] for s in i + split_token]
        x = [torch.tensor(data, dtype=torch.long).unsqueeze(0).to(device)]
        y = sample_clean(model, x, steps = block_size, temperature=0.9, sample=True, top_k=5, itos = itos, stop_token = '$', batch_size = batch_size)
        for idx in range(y.size(0)):    
            completion = y_to_completion(y[idx], itos)
            completions.append(completion)
            results.append(check_completion(completion))
            
    return results, completions

## Training loop

In [15]:
class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_tokens = 375e6 # these two numbers come from the GPT-3 paper, but may not be good defaults elsewhere
    final_tokens = 260e9 # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_path = None
    num_workers = 0 # for DataLoader

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config

        # take over whatever gpus are on the system
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)

    def save_checkpoint(self):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        logger.info("saving %s", self.config.ckpt_path)
        torch.save(raw_model.state_dict(), self.config.ckpt_path + 'model.pt')

    def train(self, writer):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, shuffle=True, pin_memory=True,
                                batch_size=config.batch_size,
                                num_workers=config.num_workers)

            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (x, y) in pbar:

                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    logits, loss = model(x, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())

                if is_train:

                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # number of tokens processed this step (i.e. label is not -100)
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens))
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")

            if not is_train:
                test_loss = float(np.mean(losses))
                logger.info("test loss: %f", test_loss)
                return test_loss
            else:
                train_loss = float(np.mean(losses))
                return train_loss

        best_loss = float('inf')
        self.tokens = 0 # counter used for learning rate decay
        for epoch in range(config.max_epochs):

            train_loss = run_epoch('train')
            writer.add_scalar('loss/train_loss',train_loss,epoch)
            if self.test_dataset is not None:
                test_loss = run_epoch('test')
                writer.add_scalar('loss/test_loss',test_loss,epoch)
                
                MolFormulas = list(self.test_dataset.data['MolFormula'].unique())    
                results, completions = check_model_output(model, MolFormulas, batch_size = 16)  
                for i in ['success','MolFormula_failure','smiles_failure','full_failure']:
                    writer.add_scalar('results/'+i,results.count(i)/len(results),epoch)

            # supports early stopping based on the test loss, or just save always if no test set is provided
            good_model = self.test_dataset is None or test_loss < best_loss
            if self.config.ckpt_path is not None and good_model:
                best_loss = test_loss
                self.save_checkpoint()


## Training

In [ ]:

# # initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=max_epochs, batch_size=batch_size, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset)*block_size,
                      num_workers=8, ckpt_path = experiment_path)
trainer = Trainer(model, train_dataset, test_dataset, tconf)
trainer.train(writer)

epoch 1 iter 991: train loss 0.31421. lr 6.000000e-04:   1%|▏         | 992/67234 [01:48<2:03:39,  8.93it/s]

TRAINS Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


epoch 1 iter 17624: train loss 0.20948. lr 5.999975e-04:  26%|██▌       | 17624/67234 [32:50<1:33:39,  8.83it/s]